In [1]:
import pickle as pkl
import numpy as np
from tqdm import tqdm
import os
import json

In [2]:
in_dir = "data/pkl_fine"
in_file = "jihualde_CV.pkl"
out_dir = "data/json/"
out_file = "data_fine.json"

In [3]:
in_path = f'{in_dir}/{in_file}'
out_path = f'{out_dir}/{out_file}'

In [3]:
def get_record(in_path):
    with open(in_path, "rb") as f_in, open(out_path, "w") as f_out:
        data = pkl.load(f_in)
        json_format = data.json_format
        lines = data.wrapper.lines
        for record in tqdm(data.record):
            # print(record)
            buf_idx = record['from']
            stk_idx = record['to']
            type_str = record['type']
            buf_string = None
            stk_string = None
            lbuf = None
            rbuf = None 
            lstk = None 
            rstk = None 
            hstk = None
            boldbuf = None
            italbuf = None
            boldstk = None
            italstk = None
            hbuf = None
            if(buf_idx == -1):
                buf_string = "$ROOT"
                lbuf = 0
                rbuf = 100
                hbuf = 30
                boldbuf = 0
                italbuf = 0
            else:
                buf_string = "$ROOT" if buf_idx == -1 else json_format[buf_idx]['text']
                lbuf = json_format[buf_idx]['x']
                rbuf = json_format[buf_idx]['x'] + json_format[buf_idx]['width']
                hbuf = int(json_format[buf_idx]['height'])
                linebuf = lines[json_format[buf_idx]['page']][json_format[buf_idx]['idx_in_page']]
                fontname = linebuf.fontname.lower()
                boldbuf = 1 if "bold" in fontname else 0
                italbuf = 1 if "italic" in fontname else 0

            if(stk_idx == -1):
                stk_string = "$ROOT"
                lstk = 0
                rstk = 100
                hstk = 30
                boldbuf = 0
                italbuf = 0
            else:
                stk_string = "$ROOT" if stk_idx == -1 else json_format[stk_idx]['text']
                lstk = json_format[stk_idx]['x']
                rstk = json_format[stk_idx]['x'] + json_format[stk_idx]['width']
                hstk = int(json_format[stk_idx]['height'])
                linebuf = lines[json_format[stk_idx]['page']][json_format[stk_idx]['idx_in_page']]
                fontname = linebuf.fontname.lower()
                boldbuf = 1 if "bold" in fontname else 0
                italbuf = 1 if "italic" in fontname else 0
            yield {
                'buf_str': buf_string,
                'lbuf': lbuf,
                'rbuf': rbuf,
                'hbuf': hbuf,
                'boldbuf': boldbuf,
                'italbuf': italbuf,
                'stk_str': stk_string,
                'lstk': lstk,
                'rstk': rstk,
                'boldstk': boldstk,
                'italstk': italstk,
                'type': type_str,
                'hstk': hstk
            }
            # print(f'{buf_string},{stk_string},{type_str}')


In [5]:
# fix a bug where the discard function had the from and to indices both point to themselves 

def fix_discard(in_path):
    
    data = None
    print(in_path)
    with open(in_path, "rb") as f_in:
        data = pkl.load(f_in)

    stack = [-1] 

    for entry in data.record:
        print("ORIGINAL", entry)
        if(entry['type'] == 'pop'):
            stack.pop()
        if(entry['type'] == 'merge'):
            stack.pop()
            stack.append(entry['from'])
        if(entry['type'] == 'subordinate'):
            stack.append(entry['from'])
        if(entry['type'] == 'discard'):
            entry['to'] = stack[-1]
        entry['to'] = 

    with open(in_path, "wb") as f_out:
        pkl.dump(data, f_out)
        # above line is pretty dangerous, should wait until everything is ready


def lengthen_sentences(in_path, out_path):
    data = None
    print(in_path)
    with open(in_path, "rb") as f_in:
        data = pkl.load(f_in)

    stack = [-1] 
    current_text = ""
    for entry in data.record:
        print("ORIGINAL", entry)
        if(entry['type'] == 'pop'):
            stack.pop()
        if(entry['type'] == 'merge'):
            stack.pop()
            stack.append(entry['from'])
        if(entry['type'] == 'subordinate'):
            stack.append(entry['from'])
        if(entry['type'] == 'discard'):
            entry['to'] = stack[-1]
        entry['to'] = 

    with open(out_path, "wb") as f_out:
        pkl.dump(data, f_out)
        # above line is pretty dangerous, should wait until everything is ready

SyntaxError: invalid syntax (1777237566.py, line 23)

In [4]:
def pkl_to_json(in_dir = "data/pkl_fine", out_dir = "data/json/", out_name = "data_fine.json"):
    to_write = {
        'buf_str': [],
        'lbuf': [],
        'rbuf': [],
        'hbuf': [],
        'boldbuf': [],
        'italbuf': [],
        'stk_str': [],
        'lstk': [],
        'rstk': [],
        'hstk': [],
        'boldstk': [],
        'italstk': [],
        'type': []
    }

    for file in os.listdir(in_dir):
        full_path = f'{in_dir}/{file}'
        print(full_path)
        if(os.path.isfile(full_path)):
            for entry in get_record(full_path):
                for key, val in entry.items():
                    to_write[key].append(val)
                    # print(key, val)
    # print(to_write)
    print(f'Found {len(to_write["buf_str"])} elements')
    out_path = f'{out_dir}/{out_name}'  

    with open(out_path, "w") as f_out:
        json.dump(to_write, f_out)

In [5]:
pkl_to_json()


FileNotFoundError: [Errno 2] No such file or directory: 'data/pkl_fine'